In [9]:
import pickle
from langchain.docstore.document import Document
import copy
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import StrOutputParser

In [2]:
def load_data_from_disk(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)

In [3]:
items_file = r"lol_chatter_backend/cached/items.pkl"
champions_file = r"lol_chatter_backend/cached/champions.pkl"
items_processed = load_data_from_disk(items_file)
champions_processed = load_data_from_disk(champions_file)

In [4]:
def item2str(item):
    name = item["name"]
    item_string = (
        f"{name} Tier: {item['tier']}\n"
        f"{name} Rank: {', '.join(item['rank'])}\n"
        f"{name} Builds From: {', '.join(item['buildsFrom']) if item['buildsFrom'] else 'None'}\n"
        f"{name} Builds Into: {', '.join(item['buildsInto']) if item['buildsInto'] else 'None'}\n"  
        f"{name} Required Champion: {item['requiredChampion'] if item['requiredChampion'] else 'None'}\n"
        f"{name} Icon: {item['icon']}\n"
        f"{name} Description: {item['simpleDescription']}\n"
        f"{name} Nicknames: {', '.join(item['nicknames']) if item['nicknames'] else 'None'}\n"
        f"{name} Passives: {str(item['passives']) if item['passives'] else 'None'}\n"
        f"{name} Active: {(item['active']) if item['active'] else 'None'}\n"
        f"{name} Stats:\n : {item['stats']}\n"
        f"{name} Shop:\n"
        f"{name} Prices Total: {item['shop']['prices']['total']}\n"
        # f"{name} Prices Combined: {item['shop']['prices']['combined']}\n"
        f"{name} Prices Sell: {item['shop']['prices']['sell']}\n"
        f"{name} Purchasable: {'Yes' if item['shop']['purchasable'] else 'No'}\n"
        f"{name} Tags: {', '.join(item['shop']['tags']) if item['shop']['tags'] else 'None'}"
    )
    item_string = item_string.replace("'", '')
    item_string = item_string.replace("\"", '')
    return item_string

for item in items_processed.values():
    item2str(item)

In [4]:


def dict2str(dct : dict) -> str:
    name = dct["name"]
    champ_string = ""
    for k in dct.keys():
        v = copy.deepcopy(dct[k])
        if isinstance(v, dict):
            for k2 in v.copy().keys():
                v[f"{name} {k2}"] = v.pop(k2)
        champ_string += f"{name} {k}: {v if v  else None}\n"

    champ_string =champ_string.replace("'", "").replace('"', "")
    return champ_string

len(dict2str(champions_processed["aatrox"]))

16359

In [5]:

docs = [Document(page_content=dict2str(item), metadata={"source":f"Item data : {item["name"]}" }) for item in items_processed.values()]

docs += [Document(page_content=dict2str(champion), metadata={"source": f"Champion data : {champion['name']}" }) for champion in champions_processed.values()]

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [106]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

vectorstore = Chroma.from_documents(
    documents=splits,  # Data
    embedding=gemini_embeddings,  # Embedding model
    persist_directory="./chroma_db2",  # Directory to save data
)



In [107]:
vectorstore_disk = Chroma(
    persist_directory="./chroma_db2",  # Directory of db
    embedding_function=gemini_embeddings,  # Embedding model
)


retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 6})
retriever.invoke("aatrox")

[Document(metadata={'source': 'Champion data : Aatrox'}, page_content='% AD, 90.000  % AD]}]}, {attribute: First Sweetspot Damage, modifiers: [{valuePerLevel: [16.000  , 40.000  , 64.000  , 88.000  , 112.000  ]}, {valuePerLevel: [96.000  % AD, 108.000  % AD, 120.000  % AD, 132.000  % AD, 144.000  % AD]}]}]}, {description: Second Cast: Aatroxs second strike affects a trapezoidal area in the target direction, with the Sweetspot at the farthest edge. The hitbox begins 100-units behind Aatrox and extends 475-units in front of him, measuring between 300 and 500-units wide from behind to in front., leveling: [{attribute: Second Cast Damage, modifiers: [{valuePerLevel: [12.500  , 31.250  , 50.000  , 68.750  , 87.500  ]}, {valuePerLevel: [75.000  % AD, 84.375  % AD, 93.750  % AD, 103.125  % AD, 112.500  % AD]}]}, {attribute: Second Sweetspot Damage, modifiers: [{valuePerLevel: [20.000  , 50.000  , 80.000  , 110.000  , 140.000  ]}, {valuePerLevel: [120.000  % AD, 135.000  % AD, 150.000  % AD, 1

In [108]:
vectorstore_disk._collection_name

'langchain'

In [85]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    # other params...
)

contextualize_q_system_prompt = (
    """
    Given a chat history and the latest user question 
    which might reference context in the chat history, 
    formulate a standalone question which can be understood 
    without the chat history. Do NOT answer the question, 
    just reformulate it if needed and otherwise return it as is.
    """
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)



In [43]:
rephrase_llm = llm
qa_llm = llm

In [91]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    """
    You are an assistant for League of Legends. You refuse to answer questions about unrelated topics.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. 
    If you can include images in your answer, do so.
    answer in details but still be concise. Be friendly and polite.
    
    
    {context}
    """
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

retriever_chain = contextualize_q_prompt | rephrase_llm | StrOutputParser() | retriever

In [92]:
from operator import itemgetter
from langchain.globals import set_debug, set_verbose
from langchain_core.runnables import RunnableBranch

set_debug(False)
set_verbose(True)

retriever_chain = contextualize_q_prompt | rephrase_llm | StrOutputParser()

chain = (
    {
        "context": RunnableBranch(
            (lambda x: x["chat_history"], retriever_chain | StrOutputParser()),  # type: ignore
            (lambda x: not x["chat_history"], itemgetter("input")),  # type: ignore
            retriever_chain | StrOutputParser(),
        )
        | retriever,
        "input": itemgetter("input"),
        "chat_history": itemgetter("chat_history"),
    }
    | qa_prompt
    | qa_llm
    | StrOutputParser()
)

# chain.invoke({"chat_history": chat_history, "input" : "What are her abilities?"})
async for chunk in chain.astream(
    {"chat_history": chat_history, "input": "What does Ahri Q do?"}
):
    print(chunk, end="", flush=True)

Ahri's Q, **Orb of Deception**, is a versatile ability that can be used for poke, farming, and securing kills. 

Here's how it works:

* **Active:** Ahri sends out an orb that deals magic damage to enemies hit.
* **Return:** At maximum range, the orb homes back to Ahri, dealing **true damage** to enemies hit on its return trip. 
* **Area of Effect:**  The orb will hit additional units around the return point in a small circle when it turns around. This applies to both the initial and return damage. 
* **Single Hit:** Each pass of the projectile can only damage an enemy once.

**Key Points:**

* **Mixed Damage:** The orb deals magic damage on the initial cast and true damage on the return.
* **True Damage:**  True damage ignores armor and magic resistance, making it very effective against tanks.
* **Range:** It has a long range, making it a good tool for poking and harassing enemies.
* **Farming:**  It can be used for farming minions, especially since it deals true damage on the return.

In [84]:
retriever_chain.invoke({"chat_history": [], "input": "How to play her"})

'Please provide more context. Who is "her"? \n'

In [72]:
chain =  itemgetter("input") | retriever

chain.invoke({"chat_history": [], "input": "Ahri q"})

[Document(metadata={'source': 'Champion data : Ahri'}, page_content='Innate:  Ahris generates a stack of Essence Fragment from killing  minions and  monsters. At max stacks, she consumes them to  heal herself., missileSpeed: None, rechargeRate: None, collisionRadius: None, tetherRadius: None, onTargetCdStatic: None, innerRadius: None, speed: None, width: None, angle: None, castTime: None, effectRadius: None, targetRange: None}], Ahri Q: [{name: Orb of Deception, icon: https://cdn.communitydragon.org/latest/champion/Ahri/ability-icon/q, effects: [{description: Active: Ahri sends her orb in the target direction that deals magic damage to enemies hit. At maximum range, the orb homes back to her to deal the same in true damage to enemies hit., leveling: [{attribute: Damage Per Pass, modifiers: [{valuePerLevel: [40.000  , 65.000  , 90.000  , 115.000  , 140.000  ]}, {valuePerLevel: [50.000  % AP, 50.000  % AP, 50.000  % AP, 50.000  % AP, 50.000  % AP]}]}, {attribute: Total Mixed Damage, modi

In [35]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "How to play ahri?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history += [
    HumanMessage(content=question),
    AIMessage(content=ai_msg_1["answer"]),
]


print(ai_msg_1["answer"])

print(chat_history)
second_question = "What her stats?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Ahri is a powerful burst mage with high mobility. Here's a breakdown of how to play her effectively:

**Understanding Ahri's Strengths:**

* **High burst damage:** Her abilities, especially her ultimate, Spirit Rush, can deal a significant amount of damage in a short time.
* **Mobility:**  Spirit Rush allows her to quickly reposition and escape dangerous situations.
* **Crowd control:** Charm can lock down enemies, setting up kills or escapes.

**Understanding Ahri's Weaknesses:**

* **Mana-intensive:** Her abilities require a lot of mana, making mana management crucial.
* **Squishy:**  She has low health and armor, making her vulnerable to ganks and sustained damage.
* **Skill-dependent:** Her abilities require accurate aiming and timing for maximum effectiveness.

**Gameplay Strategy:**

1. **Early Game:**
   * **Farm effectively:**  Focus on getting CS (Creep Score) and leveling up.
   * **Use Orb of Deception for safe harass:**  It's a good tool for poke and can be used to secure l

In [37]:
ai_msg_2["input"]

'What her stats?'

In [31]:
response = rag_chain.invoke({"input": "how to play aatrox" })
response["answer"]

"Aatrox is a powerful fighter with a high skill ceiling. Here's a breakdown of how to play him effectively:\n\n**Understanding Aatrox's Strengths**\n\n* **High Damage:** Aatrox deals massive damage, especially with his Deathbringer Stance empowered attacks and The Darkin Blade's sweet spots.\n* **Self-Sustain:** His passive and World Ender provide significant healing, making him durable in fights.\n* **Strong Late Game:** Aatrox scales well, becoming a true monster in the late game.\n* **Mobility:** Umbral Dash provides burst mobility for positioning and escaping.\n* **Crowd Control:** The Darkin Blade's knock-up and Infernal Chains' pull offer some crowd control.\n\n**Understanding Aatrox's Weaknesses**\n\n* **Mana Issues:** Aatrox is mana-dependent, requiring careful management.\n* **Early Game:** He's relatively weak early on, vulnerable to ganks and harass.\n* **Skill-Intensive:** Mastering his abilities and combos is crucial for success.\n* **Ultimate Dependence:** World Ender is 

In [51]:


messages = [
    (
        "system",
        "You are a helpful assistant for league of legends",
    ),
    ("human", "What are ahri abilities."),
    
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Ahri, the Nine-Tailed Fox, is a powerful mage with a focus on burst damage and mobility. Here's a breakdown of her abilities:\n\n**Passive:  Charm (狐火, Hǔ Huǒ)**\n\n* **Effect:**  After casting a spell, Ahri's next basic attack deals bonus magic damage and heals her for a portion of the damage dealt.\n\n**Q:  Orb of Deception (魅惑之球, Mèihuò Zhī Qiú)**\n\n* **Effect:** Ahri throws a magic orb that deals magic damage to the first enemy hit. If the orb hits an enemy champion, it bounces to another nearby enemy.\n* **Key Points:**  This ability is great for poking and farming. The bounce mechanic allows for potential AoE damage.\n\n**W:  Fox-Fire (妖狐火, Yāo Hú Huǒ)**\n\n* **Effect:** Ahri fires a projectile that deals magic damage to the first enemy hit. If the projectile hits an enemy champion, it creates a zone that damages and slows enemies within it.\n* **Key Points:**  Fox-Fire is Ahri's main damage ability. The slowing effect can be useful for setting up kills or esc

In [52]:
import google.generativeai as genai

model_1_5_flash = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction="You are a helpful assistant for league of legends",
)

model_1_5_flash.generate_content("what are ahri's abilities?")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Ahri, the Nine-Tailed Fox, is a powerful mage with a focus on mobility and burst damage. Here's a breakdown of her abilities:\n\n**Passive:  Charm (Passive)**\n\n* Ahri's basic attacks deal bonus magic damage.\n* This damage scales with Ahri's ability power.\n\n**Q: Orb of Deception (Q)**\n\n* Ahri throws a magic orb that deals magic damage to the first enemy hit.\n* The orb can be recast within a short time to make it bounce to another target, dealing slightly less damage.\n* This ability can be used for poking, wave clear, or burst damage.\n\n**W: Fox-Fire (W)**\n\n* Ahri fires three orbs of fox-fire in a cone, dealing magic damage to enemies hit.\n* If the three orbs hit the same enemy, they deal bonus magic damage. \n* This is Ahri's main poke ability an